<a href="https://colab.research.google.com/github/LeonVillanueva/CoLab/blob/master/Google_CoLab_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tensorflow==2.0.0-beta1

In [0]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, GaussianNoise, BatchNormalization, MaxPooling2D, SimpleRNN, GRU, LSTM, GlobalMaxPooling1D, Embedding
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
from scipy import stats
import math
import seaborn as sns
import re
from nltk.stem import WordNetLemmatizer
import re

> *https://www.gutenberg.org/files/4357/4357-h/4357-h.htm*

In [0]:
text = ['She flew down the stairs, found the basket and returned with it to the attic.',
        'Then she sat down before the brass-studded box and began trying one key after another in the curious old lock.',
        'Some were too large, but most were too small.']

max_vocab_size = 30000

tokenizer = Tokenizer (num_words = max_vocab_size)
tokenizer.fit_on_texts (text)
sequences = tokenizer.texts_to_sequences (text)

In [0]:
data = pad_sequences (sequences, padding='post')

> *https://www.kaggle.com/jrobischon/wikipedia-movie-plots*

In [113]:
!unzip movp.zip

Archive:  movp.zip
replace wiki_movie_plots_deduped.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
df = pd.read_csv ('wiki_movie_plots_deduped.csv')

In [115]:
df.columns.values

array(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'], dtype=object)

In [0]:
df = df[(df['Origin/Ethnicity'] == 'American') & (df['Genre'] != 'unknown')]

In [0]:
genres = df['Genre']

In [0]:
top_genres = genres.value_counts()[:10].index.tolist()

In [0]:
series = df [df['Genre'] == top_genres[0]].sample(300)

In [0]:
for i in range (1,len(top_genres)):
  series_b = df [df['Genre'] == top_genres[i]].sample(300)
  series = pd.concat((series, series_b), axis=0)

In [0]:
final = series[['Genre','Plot']]

In [0]:
lemmatizer = WordNetLemmatizer()

In [0]:
from nltk.corpus import stopwords
import nltk

In [124]:
nltk.download ('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

> *https://chrisalbon.com/machine_learning/preprocessing_text/remove_stop_words/*

> *https://h1ros.github.io/posts/stemming-words-and-sentences/*

> *https://www.datacamp.com/community/tutorials/stemming-lemmatization-python*

In [0]:
def simplify (x):
  stop_words = stopwords.words ('english')
  words = re.sub(r'[^\w\s]','',x).lower().split(' ')
  words = [lemmatizer.lemmatize (word) for word in words if word not in stop_words]
  return words

In [0]:
txt = final.iloc[0]['Plot']

In [127]:
len (simplify (txt))

64

In [128]:
final['Plot'].apply (simplify)

13098    [duchovny, play, eugene, sand, surgeon, medica...
802      [judy, page, young, society, girl, fall, love,...
11209    [joseph, megessey, known, meg, vietnam, war, v...
1930     [steel, mill, foreman, chris, bennett, pleased...
17238    [contemporary, tale, teen, whose, loyalty, tes...
                               ...                        
14823    [somewhere, distant, future, girl, living, alo...
14526    [two, engineers , aaron, abe , supplement, day...
11543    [construction, worker, douglas, quaid, troubli...
13923    [grace, stewart, occupies, remote, country, ho...
6813     [invisible, boy, mixture, lighthearted, playfu...
Name: Plot, Length: 3000, dtype: object

In [135]:
final['Genre'] = final['Genre'].map (dict (zip (list(set(final['Genre'])), list(range(0,10)))))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
X_train, X_test, y_train, y_test = train_test_split (final['Plot'], final['Genre'], test_size=0.25, random_state=23)

In [0]:
max_vocab_size = 42000

tokenizer = Tokenizer (num_words = max_vocab_size)
tokenizer.fit_on_texts (X_train)
sequence_train = tokenizer.texts_to_sequences (X_train)
sequence_test = tokenizer.texts_to_sequences (X_test)

In [0]:
word_to_idx = tokenizer.word_index

In [0]:
V = len (word_to_idx) # unique tokens

In [0]:
data_train = pad_sequences (sequence_train)
T = data_train.shape[1]
data_test = pad_sequences (sequence_test, maxlen=T, truncating='post')

In [149]:
data_train.shape, data_test.shape

((2250, 1977), (750, 1977))

In [0]:
D = 32 #Embedding Dimension

i = Input (shape=(T,))
x = Embedding (V+1, D) (i)
x = LSTM (16, return_sequences=True) (x)
x = GlobalMaxPooling1D () (x)
x = Dense (32, activation='relu') (x)
x = Dropout (0.5) (x)
x = Dense (10, activation='softmax') (x)

In [0]:
model = Model (i, x)

In [0]:
adam = tf.keras.optimizers.Adam (learning_rate=0.05, decay=5e-5)

In [0]:
model.compile (optimizer=adam,
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [0]:
epochs = 20

r = model.fit (data_train, y_train, validation_data=(data_test, y_test), epochs=epochs)

Train on 2250 samples, validate on 750 samples
Epoch 1/20
1120/2250 [=============>................] - ETA: 32s - loss: 2.3174 - accuracy: 0.1098